In [ ]:
# TODO: Clean up this notebook
import random
import datasets
from src import data_loading
from src.data_models.data_models import TrainingData

dataset = datasets.load_dataset("lmarena-ai/arena-human-preference-140k")
data = data_loading.load_training_data_lmarena(dataset["train"].to_pandas())
#dataset = datasets.load_dataset("lmsys/chatbot_arena_conversations")
#data = data_loading.load_training_data_chatbot_arena(dataset["train"].to_pandas())
#data = TrainingData(random.sample(data.entries, 10000))
print(f"Successfully loaded {len(data.entries)} entries")

In [ ]:
from src.preprocessing.length_prediction_preprocessor import LengthPredictionPreprocessor

preprocessor = LengthPredictionPreprocessor(
    embedding_model_name="all-MiniLM-L6-v2",
    min_model_comparisons=1000,
)

preprocessed = preprocessor.preprocess(data)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.preprocessing.scoring_feature_extraction import unscale_prompt_features, get_feature_descriptions
from src.preprocessing.simple_scaler import SimpleScaler
from src.analysis.feature_analysis import display_numeric_feature, display_boolean_feature

In [ ]:
lengths = SimpleScaler.from_state_dict(preprocessed.output_scaler_state).inverse_transform(
    np.array([response_length for sample in preprocessed.samples for response_length in [sample.response_length_a, sample.response_length_b]])
)
prompt_features = unscale_prompt_features(
    [sample.prompt_features for sample in preprocessed.samples], 
    SimpleScaler.from_state_dict(preprocessed.prompt_features_scaler_state))
prompt_features = [doubled for sample in prompt_features for doubled in [sample, sample]]
feature_names = get_feature_names()

In [ ]:
for feature_index in range(len(feature_names)):
    values = np.array([feature[feature_index] for feature in prompt_features])
    is_boolean = feature_index >= 27
    feature_name = feature_names[feature_index]

    print(f"--- {feature_name} ---")
    if is_boolean:
        display_boolean_feature(values, lengths, feature_name)
    else:
        display_numeric_feature(values, lengths, feature_name)
        plt.show()

In [ ]:
from src.preprocessing.scoring_feature_extraction import extract_and_transform_all_prompt_features

features, scaler = extract_and_transform_all_prompt_features(
    [entry.user_prompt for entry in data.entries],
    [entry.conversation_history for entry in data.entries],
)

In [ ]:
feature_descriptions, _ = get_feature_descriptions()
for index, feature_description in enumerate(feature_descriptions):
    values = [feature[index] for feature in features]
    sorted_values = np.sort(values)
    mean = np.mean(values)
    std = np.std(values)

    print(f"{feature_description.name}: {mean:0.2f} ± {std:0.2f} ({np.sum((np.abs(values - mean) < std)) * 100 / len(values):0.2f}% in +/- 1 std)")

    if feature_description.many_zeros:
        _, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    else:
        _, ax1 = plt.subplots(figsize=(10, 5))
        ax2 = None

    ax1.plot(sorted_values)
    ax1.axhline(mean, color="red")
    ax1.axhline(mean + std, color="orange")
    ax1.axhline(mean - std, color="orange")

    if feature_description.many_zeros:
        non_zero_values = sorted_values[sorted_values != 0]
        ax2.plot(non_zero_values)
        ax2.axhline(np.mean(non_zero_values), color="red")
        ax2.axhline(np.mean(non_zero_values) + np.std(non_zero_values), color="orange")
        ax2.axhline(np.mean(non_zero_values) - np.std(non_zero_values), color="orange")

    plt.show()